In [1]:
import numpy as np
import astropy
import ccdproc
from ccdproc import CCDData, combiner
from astropy import units as u
import matplotlib.pyplot as plt 
from matplotlib.colors import LogNorm 
from photutils.centroids import centroid_2dg 
from scipy.ndimage import shift 
from photutils import CircularAperture
from photutils import aperture_photometry
import os

In [2]:
def image_collect(directory, band):
    cwd = os.getcwd()
    os.chdir(directory)
    images = ccdproc.ImageFileCollection(".", glob_include = band)
    #defines "scim" to read the CCD data in adu from the filenames of the shifted -band images
    scim = [CCDData.read(fn, unit = "adu") for fn in images.files_filtered()]
    os.chdir(cwd)
    return scim

# Positioning of Bright Stars

In [3]:
def photometry(position, radius):
    apertures = CircularAperture(position, r = radius)
    phot_table = aperture_photometry(scim[0], apertures)
    print(phot_table)
    
    #creates a new array for all our science images photometry tables
    phot_table=[]
    
    #for loop that runs through all scim
    for idx, thisimage in enumerate(scim): 
        #calculates the flux for each star for each science image 
        phot_table.extend([aperture_photometry(thisimage, apertures)])
        print(idx, phot_table[idx], "\n")
    
    return phot_table
        
    #printing out flux values in adu for each star in first image 
    print(phot_table[0]['aperture_sum'])
    

### <font color=purple> Positions of bright stars for B, V, R and I band
<font color=purple> 

In [4]:
position_V = [(563.394,321.438), (522.479,299.957), (685.053,46.082), (259.311, 50.565), (185.395,39.328), (408.040,430.484), (415.553, 98.852), (30.863,217.533), (124.637,370.334), (517.631,174.968), (593.371,221.861)]
position_B = [(522.664,300.336), (563.243, 321.768), (185.549, 39.929), (441.632, 163.022), (429.312, 327.290), (407.941, 431.066), (429.162,327.251), (226.350,77.914), (239.797,135.189), (259.315,50.801)]  
position_R = [(563.794,321.598), (408.146,430.232), (294.930,56.057), (259.531,49.832), (30.926,217.501), (124.916,370.236), (408.212,430.140), (522.99,299.781), (49.627,27.463), (329.471,416.622), (415.754,98.519)]
#position_I

### <font color=purple> Photometry Tables & Cross checking fluxes using best ('photometric') image

In [5]:
def cross_check(best_image):
#running through all science images
    for idx, thisimage in enumerate(scim): 
        #prints the index for each image (0 is the first image in this case)
        print(idx)
        #we use our "best" image as our reference image
        print(phot_table[best_image]['aperture_sum']/phot_table[idx]['aperture_sum'])
        #calculates the median value of the flux for each image
        print(np.ma.median(phot_table[best_image]['aperture_sum']/phot_table[idx]['aperture_sum']))

In [6]:
# TypeError CCData not same units... Maybe something due to adding files other than fit? Must fix
# def combining(best_image):
#     for idx, thisimage in enumerate(scim): 
#         m = np.ma.median(phot_table[best_image]['aperture_sum'] / phot_table[idx]['aperture_sum'])
#         print(m)
#         #implement the scaling
#         scim[idx] = scim[idx].multiply(m * u.adu)

#         #calculates median for the images and combines them
#         sci_median = ccdproc.combine(scim, method = 'median',dtype = np.float32, 
#                                      minmax_clip = True, minmax_clip_min = -500)
#         del(scim)

    

# <font color=purple> Scaling and Combining each band

In [7]:
#define this...
# def photometry_median()
# print('Median combine')
# phot_table_median = aperture_photometry(sci_median, apertures)
# print('Median pixel value', np.ma.median(sci_median))
# print('Standard deviation', np.std(sci_median[400:500,400:500]))
# print(phot_table_median)
# print(phot_table[8]['aperture_sum']/phot_table_median['aperture_sum'])
# print(np.ma.median(phot_table[8]['aperture_sum']/phot_table_median['aperture_sum']))

In [8]:
scim = image_collect("../040_shift/data/shifted/march_09_2018","NGC_3201_V*")
phot_table = photometry(position_V, 10)
cross_check(12)

cwd = os.getcwd()

#for loop calculates median flux ratio for each image and displays them
for idx, thisimage in enumerate(scim): 
    m = np.ma.median(phot_table[12]['aperture_sum'] / phot_table[idx]['aperture_sum'])
    print(m)
    #implement the scaling
    scim[idx] = scim[idx].multiply(m * u.adu)

#calculates median for the images and combines them
sci_median = ccdproc.combine(scim, method = 'median',dtype = np.float32, 
                             minmax_clip = True, minmax_clip_min = -400)

os.chdir(cwd)
#converts the median combined images to a fits
sci_median.write("NGC_3201_V_median.fits")

del(scim)

apertures = CircularAperture(position_V, r = 10)
print('Median combine')
phot_table_median = aperture_photometry(sci_median, apertures)
print('Median pixel value', np.ma.median(sci_median))
print('Standard deviation', np.std(sci_median[400:500,400:500]))
print(phot_table_median)
print(phot_table[12]['aperture_sum']/phot_table_median['aperture_sum'])
print(np.ma.median(phot_table[12]['aperture_sum']/phot_table_median['aperture_sum']))

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader inste

Median combine
Median pixel value -0.790734052658081
Standard deviation 50.465797
 id xcenter ycenter    aperture_sum   
      pix     pix          adu2       
--- ------- ------- ------------------
  1 563.394 321.438 22369.942581790114
  2 522.479 299.957 29233.376587826515
  3 685.053  46.082  4778.992010974711
  4 259.311  50.565 16095.178830298008
  5 185.395  39.328  67771.73038589348
  6  408.04 430.484  30775.74116897171
  7 415.553  98.852 15415.267606227158
  8  30.863 217.533  9810.155718206186
  9 124.637 370.334  9622.549228162843
 10 517.631 174.968 28708.955591173428
 11 593.371 221.861 10355.721389627997
[1.01529964 1.02690162 1.00844542 1.01032652 1.01221011 1.01635805
 1.01823008 1.00777989 1.01270286 1.02437672 1.02149075] 1 / adu
1.0152996410680588 1 / adu


In [9]:
scim = image_collect("../040_shift/data/shifted/march_09_2018","NGC_3201_B*")
phot_table = photometry(position_B, 10)
cross_check(6)

cwd = os.getcwd()
for idx, thisimage in enumerate(scim): 
    m = np.ma.median(phot_table[6]['aperture_sum'] / phot_table[idx]['aperture_sum'])
    print(m)
    #implement the scaling
    scim[idx] = scim[idx].multiply(m * u.adu)

#calculates median for the images and combines them
sci_median = ccdproc.combine(scim, method = 'median',dtype = np.float32, 
                             minmax_clip = True, minmax_clip_min = -400)

os.chdir(cwd)
#converts the median combined images to a fits
sci_median.write("NGC_3201_B_median.fits")

del(scim)

apertures = CircularAperture(position_B, r = 10)
print('Median combine')
phot_table_median = aperture_photometry(sci_median, apertures)
print('Median pixel value', np.ma.median(sci_median))
print('Standard deviation', np.std(sci_median[400:500,400:500]))
print(phot_table_median)
print(phot_table[6]['aperture_sum']/phot_table_median['aperture_sum'])
print(np.ma.median(phot_table[6]['aperture_sum']/phot_table_median['aperture_sum']))

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader inste

Median combine
Median pixel value -0.23077520728111267
Standard deviation 10.668675
 id xcenter ycenter    aperture_sum  
      pix     pix          adu2      
--- ------- ------- -----------------
  1 522.664 300.336 6149.809363802981
  2 563.243 321.768 4804.110507302923
  3 185.549  39.929 11920.41365651146
  4 441.632 163.022 8865.445168952981
  5 429.312  327.29  6784.20306173722
  6 407.941 431.066 6206.831912135498
  7 429.162 327.251 6783.850324494577
  8  226.35  77.914  9613.93225756432
  9 239.797 135.189  5584.52093384316
 10 259.315  50.801 3617.784645514511
[1.03581395 1.04494963 1.04212701 1.02305937 1.0213336  0.9858716
 1.02124552 1.02753951 0.99730464 0.99880198] 1 / adu
1.0221964828724026 1 / adu


In [10]:
scim = image_collect("../040_shift/data/shifted/march_09_2018","NGC_3201_R*")
phot_table = photometry(position_R, 13)
cross_check(8)

cwd = os.getcwd()
for idx, thisimage in enumerate(scim): 
    m = np.ma.median(phot_table[0]['aperture_sum'] / phot_table[idx]['aperture_sum'])
    print(m)
    #implement the scaling
    scim[idx] = scim[idx].multiply(m * u.adu)

#calculates median for the images and combines them
sci_median = ccdproc.combine(scim, method = 'median',dtype = np.float32, 
                             minmax_clip = True, minmax_clip_min = -400)

os.chdir(cwd)
#converts the median combined images to a fits
sci_median.write("NGC_3201_R_median.fits")

del(scim)

apertures = CircularAperture(position_R, r = 13)
print('Median combine')
phot_table_median = aperture_photometry(sci_median, apertures)
print('Median pixel value', np.ma.median(sci_median))
print('Standard deviation', np.std(sci_median[400:500,400:500]))
print(phot_table_median)
print(phot_table[13]['aperture_sum']/phot_table_median['aperture_sum'])
print(np.ma.median(phot_table[13]['aperture_sum']/phot_table_median['aperture_sum']))

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader inste

Median combine
Median pixel value -1.2716679573059082
Standard deviation 93.69226
 id xcenter ycenter    aperture_sum   
      pix     pix          adu2       
--- ------- ------- ------------------
  1 563.794 321.598 48132.631767756364
  2 408.146 430.232  68031.15448192375
  3  294.93  56.057 16008.541732584374
  4 259.531  49.832 31856.495192883212
  5  30.926 217.501  18222.68330411645
  6 124.916 370.236  17715.36948325411
  7 408.212  430.14   68035.2570982691
  8  522.99 299.781 63159.379284704366
  9  49.627  27.463 16339.124344351221
 10 329.471 416.622  7565.854622644588
 11 415.754  98.519  35059.35550882811
[1.05949065 1.05821733 1.07654136 1.05874707 1.06244978 1.04416644
 1.05814505 1.06344203 1.06048052 1.03581588 1.05216364] 1 / adu
1.0587470713933536 1 / adu


In [ ]:
# need to do I

Best to use Median and not an average combine of an image for this case 